# Agent and Automation Function Calling BOT
A café's ordering system, agent that interacts with the user to order café drinks.

In [ ]:
%pip install -qU "google-genai>=1.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from google import genai
API_KEY = "API_KEY_KEY"
client = genai.Client(api_key=API_KEY)

## 1. Define the API
createing a  functions to track the customer's order using the global variables order (the in-progress order) and placed_order (the confirmed order sent to the kitchen). Each of the order-editing functions updates the order, and once placed, order is copied to placed_order and cleared.

In [ ]:
from typing import Optional
from random import randint

order = []
Placed_order = []

def add_to_order(drink: str, modifiers: Optional[list[str]] = None) -> None:
    """Adds the specified drink to the customer's order, including any modifiers."""
    if modifiers is None:
        modifiers = []
    order.append((drink, modifiers))


def get_order() -> list[tuple[str, list[str]]]:
    """Returns the customer's order."""
    return order


def remove_item(n: int) -> str:
    """Removes the nth (one-based) item from the order.

    Returns:
        The item that was removed.
    """
    item, _ = order.pop(n - 1)
    return item


In [ ]:
def clear_order() -> None:
    """Removes all items from the customer's order."""
    order.clear()


def confirm_order() -> str:
    """Asks the customer if the order is correct.

    Returns:
        The user's free-text response.
    """
    print("Your order:")
    if not order:
        print("  (no items)")

    for drink, modifiers in order:
        print(f"  {drink}")
        if modifiers:
            print(f'   - {", ".join(modifiers)}')

    return input("Is this correct? ")

In [ ]:
def place_order() -> int:
    """Submit the order to the kitchen.

    Returns:
        The estimated number of minutes until the order is ready.
    """
    placed_order[:] = order.copy()
    clear_order()
    return randint(1, 10)

## 2. Test the API

In [ ]:
clear_order()
add_to_order("Latte", ["Extra shot"])
add_to_order("Tea")
remove_item(2)
add_to_order("Tea", ["Earl Grey", "hot"])
confirm_order()

Your order:
  Latte
   - Extra shot
  Tea
   - Earl Grey, hot
Is this correct? yes


'yes'

## 3. Define the Prompt

In [ ]:
COFFEE_BOT_PROMPT = """\You are a coffee order taking system and you are restricted to talk only about drinks on the MENU. Do not talk about anything but ordering MENU drinks for the customer, ever.
Your goal is to do place_order after understanding the menu items and any modifiers the customer wants.
Add items to the customer's order with add_to_order, remove specific items with remove_item, and reset the order with clear_order.
To see the contents of the order so far, call get_order (by default this is shown to you, not the user)
Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will display the order items to the user and returns their response to seeing the list. Their response may contain modifications.
Always verify and respond with drink and modifier names from the MENU before adding them to the order.
If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect.
You only have the modifiers listed on the menu below: Milk options, espresso shots, caffeine, sweeteners, special requests.
Once the customer has finished ordering items, confirm_order and then place_order.

Hours: Tues, Wed, Thurs, 10am to 2pm
Prices: All drinks are free.

MENU:
Coffee Drinks:
Espresso
Americano
Cold Brew

Coffee Drinks with Milk:
Latte
Cappuccino
Cortado
Macchiato
Mocha
Flat White

Tea Drinks:
English Breakfast Tea
Green Tea
Earl Grey

Tea Drinks with Milk:
Chai Latte
Matcha Latte
London Fog

Other Drinks:
Steamer
Hot Chocolate

Modifiers:
Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free; Default option: whole
Espresso shots: Single, Double, Triple, Quadruple; default: Double
Caffeine: Decaf, Regular; default: Regular
Hot-Iced: Hot, Iced; Default: Hot
Sweeteners (option to add one or more): vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener
Special requests: any reasonable modification that does not involve items not on the menu, for example: 'extra hot', 'one pump', 'half caff', 'extra foam', etc.

"dirty" means add a shot of espresso to a drink that doesn't usually have it, like "Dirty Chai Latte".
"Regular milk" is the same as 'whole milk'.
"Sweetened" means add some regular sugar, not a sweetener.

Soy milk has run out of stock today, so soy is not available.
"""


## 4. Setup the Model

In [ ]:
from google.genai import types
from google.api_core import retry

ordering_system = [
    add_to_order,
    get_order,
    remove_item,
    clear_order,
    confirm_order,
    place_order,
]

In [ ]:
model_name = "gemini-2.0-flash-lite"  # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

chat = client.chats.create(
    model=model_name,
    config=types.GenerateContentConfig(
        tools=ordering_system,
        system_instruction=COFFEE_BOT_PROMPT,
    ),
)

placed_order = []
order = []

## 5. Chat with Bot

In [ ]:
ssfrom IPython.display import display, Markdown

print("Welcome to Barista bot!\n\n")

while not placed_order:
    response = chat.send_message(input("> "))
    display(Markdown(response.text))


print("\n\n")
print("[barista bot session over]")
print()
print("Your order:")
print(f"  {placed_order}\n")
print("- Thanks for using Barista Bot!")

Welcome to Barista bot!


> i want black hot coffee


I do not have 'black hot coffee' on the menu. Would you like an Espresso, Americano, or Cold Brew? All of these are coffee drinks served hot.

> i will go with cold brew then


Okay, I have added a Cold Brew to your order. Would you like anything else?

> do you have icream


...

> do you have long black


I do not have 'Long Black' on the menu. Would you like an Espresso, Americano, or Cold Brew?

> no, thats all
Your order:
  Cold Brew
Is this correct? yes


Okay, your order has been placed and will be ready in approximately 6 minutes.





[barista bot session over]

Your order:
  [('Cold Brew', [])]

- Thanks for using Barista Bot!
